05_softmax classification 보다 코드 더 쉽게 하는 방법

In [26]:
import numpy as np
import tensorflow as tf

In [27]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [28]:
nb_classes = 7
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
# print(Y_one_hot.shape) # (101, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_classes])
print(Y_one_hot.shape) # (101,7 )

(101, 7)


In [29]:
W = tf.Variable(tf.random.normal([16, nb_classes]), name= 'weight')
b = tf.Variable(tf.random.normal([nb_classes]), name = 'bias')
variables = [W,b]

In [30]:
def logit_fn(X):
    return tf.matmul(X,W) + b

def cost_fn(X,Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits,
                                                      from_logits=True)
    # cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    cost = tf.reduce_mean(cost_i)
    return cost

In [31]:
def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss, variables)
        return grads

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def prediction(X,Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    return accuracy

In [32]:
def fit(X,Y, epochs=500, verbose=50):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X,Y).numpy()
            loss = cost_fn(X,Y).numpy()
            print('Loss & Acc at {} epoch {}, {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)

Loss & Acc at 1 epoch 5.513110637664795, 0.12871287763118744
Loss & Acc at 50 epoch 1.1106880903244019, 0.7029703259468079
Loss & Acc at 100 epoch 0.6878463625907898, 0.801980197429657
Loss & Acc at 150 epoch 0.5162277221679688, 0.8415841460227966
Loss & Acc at 200 epoch 0.4204489588737488, 0.8514851331710815
Loss & Acc at 250 epoch 0.3559260368347168, 0.8811880946159363
Loss & Acc at 300 epoch 0.30836164951324463, 0.9108911156654358
Loss & Acc at 350 epoch 0.2716260850429535, 0.9207921028137207
Loss & Acc at 400 epoch 0.24238592386245728, 0.9603960514068604
Loss & Acc at 450 epoch 0.21858172118663788, 0.9702970385551453
Loss & Acc at 500 epoch 0.19886581599712372, 0.9702970385551453
